<a href="https://colab.research.google.com/github/winterForestStump/thesis/blob/main/notebooks/chromaDB_deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip -q install langchain chromadb sentence_transformers unstructured jq numpy==1.24.4 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.6/657.6 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 71.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.0/273.0 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
import chromadb
from langchain.retrievers import ParentDocumentRetriever
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_community.document_loaders import JSONLoader
from langchain.storage._lc_store import create_kv_docstore
from langchain.storage.file_system import LocalFileStore
import os

In [4]:
model_name = "BAAI/bge-small-en-v1.5"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cpu'},
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
# Define the metadata extraction function.
def metadata_function(record, metadata) -> dict:

    metadata["cik"] = record.get("cik")
    metadata["company"] = record.get("company")
    metadata["filing_type"] = record.get("filing_type")
    metadata["filing_date"] = record.get("filing_date")
    metadata["period_of_report"] = record.get("period_of_report")
    metadata["state_location"] = record.get("state_location")
    metadata["fiscal_year_end"] = record.get("fiscal_year_end")
    metadata["htm_filing_link"] = record.get("htm_filing_link")
    metadata["filename"] = record.get("filename")

    return metadata

In [12]:
persistent_client = chromadb.PersistentClient('/content/drive/MyDrive/Thesis/chromadb')
collection = persistent_client.get_or_create_collection("10-K")

In [13]:
fs = LocalFileStore('/content/drive/MyDrive/Thesis/store_location')
store = create_kv_docstore(fs)

# This text splitter is used to create the parent documents - The big chunks
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)

# This text splitter is used to create the child documents - The small chunks
# It should create documents smaller than the parent
child_splitter = RecursiveCharacterTextSplitter(chunk_size=256)

# The vectorstore to use to index the child chunks
vectorstore = Chroma(client = persistent_client,
                     collection_name="10-K",
                     embedding_function=bge_embeddings,
                     persist_directory='/content/drive/MyDrive/Thesis/chromadb')


In [14]:
big_chunks_retriever = ParentDocumentRetriever(
    # The underlying vectorstore to use to store small chunks and their embedding vectors
    vectorstore=vectorstore,
    # The storage interface for the parent documents
    docstore=store,
    # The text splitter to use to create child documents.
    child_splitter=child_splitter,
    # The text splitter to use to create parent documents.
    parent_splitter=parent_splitter,
    search_kwargs={'k': 2, 'search_type': 'similarity'}
)

# by default the search_type is 'similarity, also 'mmr' and 'similarity_score_threshold' are available

In [15]:
import os

directory = "/content/drive/MyDrive/Thesis/EXCTRACTED_FILINGS/3"

for filename in os.listdir(directory):
  if filename.endswith('.json'):
    file_path = os.path.join(directory, filename)
    loader = JSONLoader(file_path=file_path, jq_schema='.', content_key='content', metadata_func=metadata_function)
    data = loader.load()
    big_chunks_retriever.add_documents(data)
    print(f'The number of documents: {len(data)}')
    print(f'The number of parent docs is: {len(list(store.yield_keys()))}')

The number of documents: 1
The number of parent docs is: 10692
The number of documents: 1
The number of parent docs is: 10855
The number of documents: 1
The number of parent docs is: 11005
The number of documents: 1
The number of parent docs is: 11273


In [16]:
vectorstore.persist()